<a href="https://colab.research.google.com/github/CRM48/AIML425-A1/blob/main/Assignmnt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import sys
import numpy as np
import plotly.graph_objects as go

# Variables
seed = 2
num_inputs = 5
num_epochs = 1000

train_percent = 0.7
validation_percent = 0.15
test_percent = 0.15

# Check ratio validity
if ((train_percent + validation_percent + test_percent) != 1.0):
    print("Ratio error in data splitting")
    sys.exit(1)

# Generate Data
np.random.seed(seed)
x = np.random.normal(0, 1, (num_inputs, 3))
norms = np.sqrt(np.sum(np.square(x), axis=1, keepdims=True))
y = x / norms

# Split Data



print("Input x:")
print(x)

print("Target y (on unit sphere):")
print(y)

print("Target norms")
print(norms)

"""


fig = go.Figure()

# Add normalized y points in blue
fig.add_trace(go.Scatter3d(
    x=y[:, 0], y=y[:, 1], z=y[:, 2],
    mode='markers',
    marker=dict(size=3, color='blue'),
    name='Normalized y'
))

fig.update_layout(
    title="Original Points (red) and Normalized Points on Unit Sphere (blue)",
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    )
)

fig.show()
"""

Ratio error in data splitting


SystemExit: 1

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

